In [ ]:
import os
import pandas as pd

path = '../input/ventilator-pressure-prediction'
df_train = pd.read_csv(os.path.join(path, 'train.csv'))
df_test = pd.read_csv(os.path.join(path, 'test.csv'))
# sub = pd.read_csv(os.path.join(path, 'sample_submission.csv'))

In [ ]:
submission = pd.read_csv('../input/ventilator-pressure-prediction/sample_submission.csv')

In [ ]:
df_test.head(10)

EDA

In [ ]:
# col = list(df_train.columns)
# col.remove('id')
# col.remove('breath_id')

# df_train[col].describe().T

In [ ]:
# R indicates how restricted the airway is
df_train.R.unique()

In [ ]:
#C indicates how compliant the lung is
df_train.C.unique()

In [ ]:
import seaborn as sns

sns.boxplot(data=df_train[['u_in','u_out','pressure']])

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

col_list = ['R', 'C']

fig, axes = plt.subplots(1, len(col_list), figsize=(10,8))

for i, col in enumerate(col_list):
    sns.boxplot(data=df_train, y='pressure', x=col, ax=axes[i])


In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

fig = plt.figure(figsize = (13, 8))
rc = ['R', 'C']
for i in rc:
    plt.subplot(2, 2, rc.index(i)+1)
    plt.title(i, y = 1.2, size = 25, fontname = 'monospace', color = 'black')
    a = sns.countplot(x = i, data = df_train, palette = ['#48871b', '#c45f4b', '#3c4e91'])
    plt.ylabel('')
    plt.xlabel('')
    plt.xticks(fontname = 'monospace', size = 12)
    plt.yticks([])
    for j in ['right', 'top']:
        a.spines[j].set_visible(False)
    for j in ['bottom', 'left']:    
        a.spines[j].set_linewidth(1.2)
        
    summ = 0
    for p in a.patches:
        summ += p.get_height()

    for p in a.patches:
        height = p.get_height()
        a.annotate(f'{height}', (p.get_x() + p.get_width() / 2, p.get_height()), 
                   ha = 'center', va = 'center', 
                   size = 13,
                   xytext = (1, -15), 
                   textcoords = 'offset points',
                   fontname = 'monospace', color = 'white')
        a.annotate(f'{round((height/summ) * 100, 1)}%', (p.get_x() + p.get_width() / 2, p.get_height()), 
                   ha = 'center', va = 'center', 
                   size = 15,
                   xytext = (1, 13), 
                   textcoords = 'offset points',
                   fontname = 'monospace', color = 'black')   
        
for i in rc:
    plt.subplot(2, 2, rc.index(i)+3)
    a = sns.countplot(x = i, 
                      data = df_test, 
                      palette = ['#48871b', '#c45f4b', '#3c4e91'])
    plt.ylabel('')
    plt.xlabel('')
    plt.xticks(fontname = 'monospace', size = 12)
    plt.yticks([])
    for j in ['right', 'top']:
        a.spines[j].set_visible(False)
    for j in ['bottom', 'left']:    
        a.spines[j].set_linewidth(1.2)
        
    summ = 0
    for p in a.patches:
        summ += p.get_height()

    for p in a.patches:
        height = p.get_height()
        a.annotate(f'{height}', (p.get_x() + p.get_width() / 2, p.get_height()), 
                   ha = 'center', va = 'center', 
                   size = 13,
                   xytext = (1, -15), 
                   textcoords = 'offset points',
                   fontname = 'monospace', color = 'white')
        a.annotate(f'{round((height/summ) * 100, 1)}%', (p.get_x() + p.get_width() / 2, p.get_height()), 
                   ha = 'center', va = 'center', 
                   size = 15,
                   xytext = (1, 13), 
                   textcoords = 'offset points',
                   fontname = 'monospace', color = 'black')
        
plt.figtext(0.15, 
            1.1, 
            'Distribution of lung attributes (R/C)',
            fontname = 'monospace', 
            size = 30, 
            color = 'black')

plt.figtext(1.03, 0.15, 
            'TEST', 
            fontname = 'monospace',
            size = 25,
            color = 'black',
            rotation = 90)

plt.figtext(1.03,
            0.7,
            'TRAIN',
            fontname = 'monospace',
            size = 25,
            color = 'black',
            rotation = 90)
        
fig.tight_layout(h_pad = 10)
plt.show()

In [ ]:
r, c, plot = [5, 20, 50], [10, 20, 50], 1
fig = plt.figure(figsize = (15, 12))
plot = 1
for i in range(3):
    rr = r[i]
    for k in range(3):
        cc = c[k]
        plt.subplot(3, 3, plot)
        plt.title(f'R = {rr} | C = {cc}', 
                  fontname = 'monospace',
                  size = 15, 
                  color = 'black')
        
        a = sns.kdeplot(df_train.query('time_step < 0.000001 & u_in < 0.000001 & R == @rr & C == @cc')['pressure'], 
                        color = '#48871b', 
                        shade = True, 
                        alpha = 1, 
                        linewidth = 1.5, 
                        edgecolor = 'black')
        plt.ylabel('')
        plt.xlabel('')
        plt.xticks(size = 12, fontname = 'monospace')
        plt.yticks([])

        for j in ['right', 'top']:
            a.spines[j].set_visible(False)
        for j in ['bottom', 'left']:    
            a.spines[j].set_linewidth(1.2)
            
        plot += 1

y = 1.27
for i in range(3):
    rr = r[i]
    y -= 0.333
    x = -0.315
    for k in range(3):
        cc = c[k]
        x += 0.333
        plt.figtext(x,
                    y, 
                    f'Min: {round(df_train.query("time_step < 0.000001 & u_in < 0.000001 & R == @rr & C == @cc")["pressure"].min(),2)}', 
                    fontname = 'monospace', 
                    color = 'black')
        
        plt.figtext(x,
                    y-0.02, 
                    f'Max: {round(df_train.query("time_step < 0.000001 & u_in < 0.000001 & R == @rr & C == @cc")["pressure"].max(),2)}',
                    fontname = 'monospace')
        
        plt.figtext(x,
                    y-0.04,
                    f'Mean: {round(df_train.query("time_step < 0.000001 & u_in < 0.000001 & R == @rr & C == @cc")["pressure"].mean(),2)}',
                    fontname = 'monospace',
                    color = 'black')
        
        plt.figtext(x,
                    y-0.06, 
                    f'Median: {round(df_train.query("time_step < 0.000001 & u_in < 0.000001 & R == @rr & C == @cc")["pressure"].median(),2)}', 
                    fontname = 'monospace', 
                    color = 'black')
        
plt.figtext(0.01, 1.08, 'presssure distribution depending on lung attributes', fontname = 'monospace', size = 30, color = 'black')
        
fig.tight_layout(h_pad = 3)
plt.show()

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd

def pressure_eda (data: pd.DataFrame, 
                  col_name: str, 
                  lung_state: list):
    
    fig, axes = plt.subplots(1, len(lung_state), figsize=(10,8))
        
    for i, state in enumerate(lung_state):
        mask = data[col_name] == state
        df = data.loc[mask]
        sns.scatterplot(data=df, x='time_step', y='pressure', ax=axes[i])
        axes[i].set_title(f'{col_name} = {state}')

lung_state_c = list(df_train.C.unique())
pressure_eda(df_train, 'C', lung_state_c)

In [ ]:
lung_state_r = list(df_train.R.unique())
pressure_eda(df_train, 'R', lung_state_r)

In [ ]:
sns.scatterplot(data=df_train, x='time_step', y='pressure')

In [ ]:
df_train['breath_id'].value_counts().describe()

In [ ]:
df_train.breath_id.unique()

In [ ]:
import tensorflow as tf

from sklearn.preprocessing import RobustScaler, normalize
from tensorflow.keras.callbacks import EarlyStopping

In [ ]:
def features(df):
    df['area'] = df['time_step'] * df['u_in']
    df['area'] = df.groupby('breath_id')['area'].cumsum()
    
    df['u_in_cumsum'] = (df['u_in']).groupby(df['breath_id']).cumsum()
    
    df['u_in_lag2'] = df['u_in'].shift(2).fillna(0)
    df['u_in_lag4'] = df['u_in'].shift(4).fillna(0)
    
    df['R'] = df['R'].astype(str)
    df['C'] = df['C'].astype(str)
    df = pd.get_dummies(df)
    
    df['ewm_u_in_mean'] = df.groupby('breath_id')['u_in'].ewm(halflife=10).mean().reset_index(level=0,drop=True)
    df['ewm_u_in_std'] = df.groupby('breath_id')['u_in'].ewm(halflife=10).std().reset_index(level=0,drop=True)
    df['ewm_u_in_corr'] = df.groupby('breath_id')['u_in'].ewm(halflife=10).corr().reset_index(level=0,drop=True)
    
    df['rolling_10_mean'] = df.groupby('breath_id')['u_in'].rolling(window=10, min_periods=1).mean().reset_index(level=0,drop=True)
    df['rolling_10_max'] = df.groupby('breath_id')['u_in'].rolling(window=10, min_periods=1).max().reset_index(level=0,drop=True)
    df['rolling_10_std'] = df.groupby('breath_id')['u_in'].rolling(window=10, min_periods=1).std().reset_index(level=0,drop=True)
    
    df['expand_mean'] = df.groupby('breath_id')['u_in'].expanding(2).mean().reset_index(level=0,drop=True)
    df['expand_max'] = df.groupby('breath_id')['u_in'].expanding(2).max().reset_index(level=0,drop=True)
    df['expand_std'] = df.groupby('breath_id')['u_in'].expanding(2).std().reset_index(level=0,drop=True)
    
    return df

train = features(df_train)
test = features(df_test)

In [ ]:
train = train.fillna(0)
test = test.fillna(0)

targets = train[['pressure']].to_numpy().reshape(-1, 80)
train.drop(['pressure', 'id', 'breath_id'], axis = 1, inplace = True)
test = test.drop(['id', 'breath_id'], axis = 1)

In [ ]:
#scaler
from sklearn.preprocessing import RobustScaler

RS = RobustScaler()
train_scaled = RS.fit_transform(train)
test_scaled = RS.transform(test)

In [ ]:
train_scaled = train_scaled.reshape(-1, 80, train.shape[-1])
test_scaled = test_scaled.reshape(-1, 80, train.shape[-1])

In [ ]:
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(train_scaled, targets, train_size=0.8)


In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, RNN, Bidirectional, Input

from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping

from sklearn.model_selection import KFold


EPOCH = 300
BATCH_SIZE = 1024

model = Sequential()
model.add(Input(shape= train.shape[-2:]))
model.add(Bidirectional(LSTM(200, return_sequences=True)))
model.add(Bidirectional(LSTM(100, return_sequences=True)))
model.add(Dense(50, activation='relu'))
model.add(Dense(1))

model.compile(optimizer=Adam(learning_rate=0.001), 
              loss = "mae",)

model.fit(x_train, 
          y_train, 
          validation_data = (x_test, y_test), 
          epochs = EPOCH, 
          batch_size = BATCH_SIZE, 
          callbacks = [EarlyStopping(monitor='val_mae', patience=20)])


In [ ]:
# pred= model.predict(test_scaled).squeeze().reshape(-1, 1).squeeze()

# submission["pressure"] = pred
# submission.to_csv('submission.csv', index=False)

In [ ]:
# !pip install git+https://github.com/keras-team/keras-tuner.git -q
# import keras_tuner as kt

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Layer, Dropout, Input
from tensorflow.keras.layers import Bidirectional, LSTM

from tensorflow.keras.optimizers import Adam



EPOCH = 300
BATCH_SIZE = 1024

def model_builder(hp):

    model = Sequential()
    model.add(Input(shape= train.shape[-2:]))
    
    for i in range(hp.Int('n_layers', 1, 4)):
        model.add(Bidirectional(LSTM(hp.Int(f'lstm_{i}_units',min_value=100,max_value=500,step=100),return_sequences=True)))

    model.add(Dropout(hp.Float('Dropout_rate',min_value=0,max_value=0.5,step=0.1)))
    
    model.add(Dense(hp.Int('dense_units',min_value=100, max_value=500, step=100), activation=hp.Choice('dense_activation',values=['relu', 'selu'],default='relu')))
    
#     model.add(Dense(50, activation='relu'))
    model.add(Dense(1))
    
    model.compile(loss='mae', 
                  optimizer=Adam(hp.Choice('learning_rate', values=[1e-2, 1e-3, 1e-4])),
                  metrics = ['mae'])

    return model 
        
tuner = kt.Hyperband(model_builder, 
                     objective='val_mae',
                     max_epochs=10 
                    )

tuner.search(
            x=x_train,
            y=y_train,
            epochs=EPOCH,
            batch_size=BATCH_SIZE,
            validation_data=(x_test,y_test),
            )

In [ ]:
# # Get the optimal hyperparameters
# best_hps = tuner.get_best_hyperparameters(num_trials=1)[0]

# # print(f"""
# # Best Hyperparameters: 
# # - best number of fuly connected layers is {best_hps.get('n_layers')}
# # - optimal number of units in the first LSTM layer is {best_hps.get('lstm_0_units')} 
# # - optimal number of units in the first LSTM layer is {best_hps.get('lstm_1_units')} 
# # - optimal number of units in the first LSTM layer is {best_hps.get('lstm_2_units')}
# # - optimal number of units in the first LSTM layer is {best_hps.get('lstm_3_units')} 
# # - optimal number of units in the first densely-connected layer is {best_hps.get('dense_units')} 
# # - best activation function is {best_hps.get('dense_activation')}
# # - optimal learning rate for the optimizer is {best_hps.get('learning_rate')}
# # - optimal dropout is {best_hps.get('Dropout_rate')}
# # """)

In [ ]:
from tensorflow.keras.callbacks import EarlyStopping

best_hps = tuner.get_best_hyperparameters(num_trials=1)[0]

#train of the model
kt_model = tuner.hypermodel.build(best_hps)
kt_history = kt_model.fit(x=x_train,
                          y=y_train,
                          epochs=300, 
                          validation_data=(x_test,y_test), 
                          callbacks = [EarlyStopping(monitor='val_mae', patience=20)]
                         )

kt_val_mae_per_epoch = kt_history.history['val_mae']
best_epoch = kt_val_mae_per_epoch.index(max(kt_val_mae_per_epoch)) + 1

In [ ]:
# kt_hypermodel = tuner.hypermodel.build(best_hps)

# #retrain the model with the optimal number of epochs
# kt_hypermodel.fit(x=x_train,y=y_train, epochs=best_epoch)

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, RNN, Bidirectional, Input, Dropout

from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping

from sklearn.model_selection import KFold


EPOCH = 300
BATCH_SIZE = 1024

model = Sequential()
model.add(Input(shape= train.shape[-2:]))
model.add(Bidirectional(LSTM(200, return_sequences=True)))
model.add(Bidirectional(LSTM(200, return_sequences=True)))
model.add(Bidirectional(LSTM(500, return_sequences=True)))
model.add(Bidirectional(LSTM(400, return_sequences=True)))
model.add(Dropout(0.3))
model.add(Dense(400, activation='selu'))
model.add(Dense(1))

model.compile(optimizer=Adam(learning_rate=0.001), 
              loss = "mae",)

model.fit(x_train, 
          y_train, 
          validation_data = (x_test, y_test), 
          epochs = 183, 
          batch_size = BATCH_SIZE, 
          callbacks = [EarlyStopping(monitor='val_mae', patience=20)])


In [ ]:
pred= model.predict(test_scaled).squeeze().reshape(-1, 1).squeeze()

submission["pressure"] = pred
submission.to_csv('submission.csv', index=False)